In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 4"

In [2]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
import optuna
import gc
xgb.__version__

'1.6.2'

In [3]:
!nvidia-smi

Tue Nov 22 19:15:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0  On |                    0 |
| N/A   41C    P0    39W / 300W |    359MiB / 32499MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   

In [4]:
!nvidia-smi -L

GPU 0: Tesla V100-DGXS-32GB (UUID: GPU-d752af09-1f62-bf3b-4f70-78b84e9e41f6)
GPU 1: Tesla V100-DGXS-32GB (UUID: GPU-054a4a35-f98a-3ebc-9100-0f697c246b43)
GPU 2: Tesla V100-DGXS-32GB (UUID: GPU-454525d4-bebd-7fb8-0ba3-3b85e2f99abd)
GPU 3: Tesla V100-DGXS-32GB (UUID: GPU-af0fb74e-f5eb-0833-17ff-494cc6bdcee1)


In [5]:
train_x = dd.read_csv('../input/higgs_small_roc/xgtrain_25_3.csv')
test_x = dd.read_csv('../input/higgs_small_roc/xgval.csv')

In [6]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [7]:
cluster = LocalCUDACluster(n_workers=4)
client = Client(cluster)

In [8]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)

In [9]:
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)


In [10]:
num_round = 1000

In [11]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']),
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 3,25),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()

    roc = roc_auc_score(test_y, predictions)
    
    return roc

In [12]:
study = optuna.create_study(direction='maximize')

[I 2022-11-22 19:15:57,975] A new study created in memory with name: no-name-0edc52dc-a426-4d11-b222-3fad700cba43


In [13]:
%%time
study.optimize(objective, n_trials=2)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[I 2022-11-22 19:16:13,868] Trial 0 finished with value: 0.8094874183304518 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.056071448038671784, 'alpha': 1.9019092519236398, 'colsample_bytree': 0.5490627208494752, 'subsample': 0.5562034008141327, 'learning_rate': 0.042093909972864975, 'max_depth': 9, 'min_child_weight': 233, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.8094874183304518.
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[I 2022-11-22 19:17:17,710] Trial 1 finished with value: 0.8064716488645771 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 9.414737387351568, 'alpha': 0.0069980

CPU times: user 8.83 s, sys: 2.67 s, total: 11.5 s
Wall time: 1min 19s


In [14]:
%%time
study.optimize(objective, n_trials=250)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[I 2022-11-22 19:17:51,022] Trial 2 finished with value: 0.8131713694733884 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.13262777006712875, 'alpha': 0.0010968738856912688, 'colsample_bytree': 0.7725257476589437, 'subsample': 0.647021024558522, 'learning_rate': 0.01791516877631808, 'max_depth': 14, 'min_child_weight': 271, 'eval_metric': 'logloss'}. Best is trial 2 with value: 0.8131713694733884.
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[I 2022-11-22 19:18:19,869] Trial 3 finished with value: 0.7867546720212514 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 1.146089753070209, 'alpha': 0.066577

CPU times: user 1h 11min 20s, sys: 16min 16s, total: 1h 27min 37s
Wall time: 13h 29min 35s


In [15]:
study.best_trial.params

{'objective': 'binary:logistic',
 'tree_method': 'gpu_hist',
 'lambda': 0.487925669934271,
 'alpha': 0.02968106363177915,
 'colsample_bytree': 0.8316132965578497,
 'subsample': 0.6513397111981933,
 'learning_rate': 0.017195258162539133,
 'max_depth': 23,
 'min_child_weight': 274,
 'eval_metric': 'logloss'}

In [16]:
real_test_x = dd.read_csv('../input/higgs_small_roc/xgtest.csv')

In [17]:
real_test_y = real_test_x['target']
real_test_x = real_test_x[real_test_x.columns.difference(['target'])]

In [18]:
drealtest = xgb.dask.DaskDMatrix(client, real_test_x, real_test_y)

In [19]:
%%time
params = study.best_trial.params

output = xgb.dask.train(client, params, dtrain, num_round)
    
booster = output['booster']  # booster is the trained model
booster.set_param({'predictor': 'gpu_predictor'})

predictions = xgb.dask.predict(client, booster, drealtest)
    
predictions = predictions.compute()

roc_auc_score(real_test_y, predictions)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


CPU times: user 13.1 s, sys: 2.81 s, total: 15.9 s
Wall time: 2min 26s


0.8068979385996837

In [20]:
%%time 
scores = []

for jj in range(15):
    print(jj)
    params = study.best_trial.params
    params['seed'] = 5*jj**3 + 137

    output = xgb.dask.train(client, params, dtrain, num_round)

    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, drealtest)

    predictions = predictions.compute()
    score = roc_auc_score(real_test_y, predictions)
    print(score)

    scores.append(score)

0


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


0.8063484910032072
1


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


0.8062360196786988
2


...

0.8063938969631178
13


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


0.8068374112341399
14


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


0.8067598944632332
CPU times: user 3min 9s, sys: 46.5 s, total: 3min 55s
Wall time: 36min 15s


In [21]:
import numpy as np
np.mean(scores)

0.8065866474022673

In [22]:
scores

[0.8063484910032072,
 0.8062360196786988,
 0.8066813810779454,
 0.8064821573414086,
 0.8063657047772528,
 0.8068816536076266,
 0.806654070713182,
 0.8063837638472654,
 0.8064824808497415,
 0.8068242778393921,
 0.8066332252972037,
 0.8068352823405932,
 0.8063938969631178,
 0.8068374112341399,
 0.8067598944632332]